In [1]:
import json
import pandas as pd

In [2]:
N = 5000
SIM = .94 # 100% correct: struct sim .999
HASH = 3
path = "../data/tablebank/Detection_data/"

## Latex

In [3]:
data = None
with open(path + "Latex/Latex.json") as json_file:
    data = json.load(json_file)

In [8]:
#data.keys()

In [5]:
def filter_special_filenames(x):
    x = x[list(x.keys())[0]]["file_name"]
    for a in x:
        if ord(a) > 200:
            return False
    return True

In [6]:
file_ids = list(filter(lambda x: filter_special_filenames(x), 
                       map(lambda x: {x["id"]: {"file_name": x['file_name'],
                                                "width": x['width'], "height": x['height']}}, 
                    data["images"])))[0:N]
file_ids = {k: v for d in file_ids for k, v in d.items()}
print(len(file_ids))
#print(file_ids)

5000


In [7]:
annotations = list(filter(lambda x: x[0] in file_ids, 
                          map(lambda x: [x["image_id"], x["segmentation"]], data["annotations"])))
annotations_dict = {}
for annotation in annotations:
    if annotation[0] not in annotations_dict:
        annotations_dict[annotation[0]] = []
    annotations_dict[annotation[0]].append(annotation[1])
annotations = annotations_dict
#annotations

In [8]:
df = pd.DataFrame({"id": [], "file": [], "format": [], "width": [], "height": [],
                   "bounding_box": []})
df

,id,file,format,width,height,bounding_box


In [9]:
for x in file_ids:
    file = {"id": x, 
            "file": path + "Latex/images/" + file_ids[x]["file_name"], 
            "format": "Latex", 
            "width": file_ids[x]["width"],
            "height": file_ids[x]["height"], 
            "bounding_box": annotations[x]}
    df = df.append(file, ignore_index = True)

In [3]:
#df.head()

## Word

In [11]:
data = None
with open(path + "Word/Word.json") as json_file:
    data = json.load(json_file)

In [12]:
file_ids = list(filter(lambda x: filter_special_filenames(x), 
                       map(lambda x: {x["id"]: {"file_name": x['file_name'],
                                                "width": x['width'], "height": x['height']}}, 
                    data["images"])))[0:N]
file_ids = {k: v for d in file_ids for k, v in d.items()}
#file_ids
len(file_ids)

5000

In [13]:
annotations = list(filter(lambda x: x[0] in file_ids, 
                          map(lambda x: [x["image_id"], x["segmentation"]], data["annotations"])))
annotations_dict = {}
for annotation in annotations:
    if annotation[0] not in annotations_dict:
        annotations_dict[annotation[0]] = []
    annotations_dict[annotation[0]].append(annotation[1])
annotations = annotations_dict
#annotations

In [14]:
for x in file_ids:
    file = {"id": x, 
            "file": path + "Word/images/" + file_ids[x]["file_name"], 
            "format": "Word", 
            "width": file_ids[x]["width"],
            "height": file_ids[x]["height"], 
            "bounding_box": annotations[x]}
    df = df.append(file, ignore_index = True)

In [4]:
#df.tail()

In [16]:
df.to_csv("../data/tablebank/Detection_data/gt_5000.csv", sep=";", index=0)

## GT as PDF

### Latex

In [3]:
import pandas as pd
import urllib
from PyPDF2 import PdfFileWriter, PdfFileReader
import os
import docx2pdf
from pdf2image import convert_from_path
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
import PIL
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim
import time
from io import BytesIO
import imagehash
from shutil import copyfile
import signal

In [5]:
# TODO uncomment
# gt_subset = pd.read_csv("../data/tablebank/Detection_data/gt_5000.csv", sep=";", index_col=0)
# gt_subset.head()

In [5]:
def get_file_name(url):
    parts = url.split("/")
    return parts[len(parts)-1]

In [6]:
# TODO uncomment
# latex_links = pd.read_csv("../data/tablebank/Detection_data/Latex/url.csv", sep=",")
# latex_links = latex_links[latex_links["image_name"].isin(gt_subset["file"].apply(lambda x: get_file_name(x)))]
#latex_links.head()

In [6]:
def compare_img_s(img1, img2):
    # imgA = img1.convert('1')
    # imgB = img2.convert('1')
    imgA = np.array(img1, dtype=np.uint8)
    imgB = np.array(img2, dtype=np.uint8)
    sim = ssim(imgA, imgB, multichannel=True)
    return sim

def compare_img_mse(img1, img2):
    imgA = img1.convert('1')
    imgB = img2.convert('1')
    imgA = np.array(imgA, dtype=np.uint8)
    imgB = np.array(imgB, dtype=np.uint8)
    squared_diff = (imgA - imgB) ** 2
    summed = np.sum(squared_diff)
    num_pix = imgA.shape[0] * imgA.shape[1] #img1 and 2 should have same shape
    err = summed / num_pix
    return err

def compare_img(img1, img2):
    return imagehash.average_hash(img1) - imagehash.average_hash(img2)

In [7]:
s = []
def get_files_latex(url_row, err, s):
    # get
    filename = get_file_name(url_row["image_name"].replace("jpg", "pdf"))
    e = False
    if not os.path.exists("../data/tablebank/Detection_data/Latex/pdf/" + filename):
        try:
            data = urllib.request.urlretrieve(url_row["url"], "test.pdf")
            inputpdf = PdfFileReader(open("test.pdf", "rb"))
            # store
            output = PdfFileWriter()
            output.addPage(inputpdf.getPage(url_row["page"]))
            file1 = "../data/tablebank/Detection_data/Latex/pdf/" + get_file_name(url_row["image_name"].
                                                                                  replace("jpg", "pdf"))
            with open("../data/tablebank/Detection_data/Latex/pdf/" + filename, "wb") as outputStream:
                output.write(outputStream)
        except Exception:
            e = True
    # compare
    file1 = "../data/tablebank/Detection_data/Latex/pdf/" + filename
    file2 = "../data/tablebank/Detection_data/Latex/images/" + url_row["image_name"]
    img2 = Image.open(file2)
    try:
        img1 = convert_from_path(file1, size=img2.size)[0]
        img1 = img1.resize(img2.size)
        sim = compare_img(img1, img2)
        print(file1 + " " + str(sim))
        if sim > s:
            e = True
    except Exception:
        e = True
    if e:
        err.append(file1)

In [23]:
err_latex = []
latex_links.apply(lambda x: get_files_latex(x, err_latex, HASH), axis=1)
print(len(err_latex))

../data/tablebank/Detection_data/Latex/pdf/1401.0007_15.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1401.0007_5.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1401.0016_5.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1401.0022_16.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1401.0023_10.pdf 24
../data/tablebank/Detection_data/Latex/pdf/1401.0023_15.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1401.0023_17.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1401.0023_2.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1401.0023_9.pdf 12
../data/tablebank/Detection_data/Latex/pdf/1401.0033_1.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1401.0033_3.pdf 19
../data/tablebank/Detection_data/Latex/pdf/1401.0045_3.pdf 8
../data/tablebank/Detection_data/Latex/pdf/1401.0045_4.pdf 19
../data/tablebank/Detection_data/Latex/pdf/1401.0045_6.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1401.0047_2.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1401.0047_3.pdf 6
../data/tableba

../data/tablebank/Detection_data/Latex/pdf/1401.0332_3.pdf 11
../data/tablebank/Detection_data/Latex/pdf/1401.0332_4.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1401.0332_5.pdf 8
../data/tablebank/Detection_data/Latex/pdf/1401.0339_20.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.0362_5.pdf 9
../data/tablebank/Detection_data/Latex/pdf/1401.0363_3.pdf 6
../data/tablebank/Detection_data/Latex/pdf/1401.0369_6.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.0380_26.pdf 9
../data/tablebank/Detection_data/Latex/pdf/1401.0385_5.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.0391_10.pdf 9
../data/tablebank/Detection_data/Latex/pdf/1401.0391_8.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.0391_9.pdf 9
../data/tablebank/Detection_data/Latex/pdf/1401.0404_25.pdf 28
../data/tablebank/Detection_data/Latex/pdf/1401.0404_26.pdf 7
../data/tablebank/Detection_data/Latex/pdf/1401.0404_27.pdf 8
../data/tablebank/Detection_data/Latex/pdf/1401.0404_28.pdf 4
../data/tableba

../data/tablebank/Detection_data/Latex/pdf/1401.0729_16.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.0729_17.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.0729_18.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.0729_19.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.0729_20.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.0729_21.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1401.0729_22.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.0729_23.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.0733_8.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.0750_10.pdf 6
../data/tablebank/Detection_data/Latex/pdf/1401.0750_11.pdf 7
../data/tablebank/Detection_data/Latex/pdf/1401.0750_12.pdf 5
../data/tablebank/Detection_data/Latex/pdf/1401.0750_13.pdf 7
../data/tablebank/Detection_data/Latex/pdf/1401.0750_14.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1401.0750_8.pdf 9
../data/tablebank/Detection_data/Latex/pdf/1401.0750_9.pdf 6
../data/tab

../data/tablebank/Detection_data/Latex/pdf/1401.1105_5.pdf 11
../data/tablebank/Detection_data/Latex/pdf/1401.1113_4.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.1119_3.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1401.1124_13.pdf 9
../data/tablebank/Detection_data/Latex/pdf/1401.1124_14.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1401.1124_5.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1401.1124_6.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1401.1124_7.pdf 16
../data/tablebank/Detection_data/Latex/pdf/1401.1124_8.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1401.1124_9.pdf 8
../data/tablebank/Detection_data/Latex/pdf/1401.1127_4.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1401.1127_5.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1401.1134_2.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1401.1135_3.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1401.1145_13.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1401.1145_15.pdf 2
../data/tablebank/

../data/tablebank/Detection_data/Latex/pdf/1401.1406_4.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1401.1406_6.pdf 9
../data/tablebank/Detection_data/Latex/pdf/1401.1406_9.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1401.1411_1.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1401.1411_3.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1401.1412_10.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1401.1412_16.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1401.1412_24.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1401.1412_26.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1401.1412_33.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1401.1414_5.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1401.1425_6.pdf 6
../data/tablebank/Detection_data/Latex/pdf/1401.1437_16.pdf 5
../data/tablebank/Detection_data/Latex/pdf/1401.1437_18.pdf 5
../data/tablebank/Detection_data/Latex/pdf/1401.1437_23.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1401.1437_28.pdf 1
../data/tableba

...

../data/tablebank/Detection_data/Latex/pdf/1402.3272_5.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.3272_6.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.3272_9.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1402.3292_3.pdf 10
../data/tablebank/Detection_data/Latex/pdf/1402.3293_11.pdf 5
../data/tablebank/Detection_data/Latex/pdf/1402.3295_6.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1402.3299_2.pdf 7
../data/tablebank/Detection_data/Latex/pdf/1402.3299_9.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1402.3300_10.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.3300_16.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.3300_27.pdf 10
../data/tablebank/Detection_data/Latex/pdf/1402.3300_44.pdf 7
../data/tablebank/Detection_data/Latex/pdf/1402.3305_8.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1402.3312_4.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1402.3312_5.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1402.3312_7.pdf 2
../data/tablebank

../data/tablebank/Detection_data/Latex/pdf/1402.3630_0.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1402.3630_1.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1402.3634_8.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1402.3635_14.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1402.3635_15.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1402.3637_11.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.3637_16.pdf 26
../data/tablebank/Detection_data/Latex/pdf/1402.3637_17.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.3639_4.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1402.3649_12.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1402.3649_13.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1402.3659_14.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1402.3669_4.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1402.3670_4.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1402.3671_2.pdf 8
../data/tablebank/Detection_data/Latex/pdf/1402.3671_4.pdf 5
../data/tableba

../data/tablebank/Detection_data/Latex/pdf/1402.3989_3.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1402.4003_8.pdf 5
../data/tablebank/Detection_data/Latex/pdf/1402.4017_2.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1402.4025_5.pdf 5
../data/tablebank/Detection_data/Latex/pdf/1402.4025_6.pdf 9
../data/tablebank/Detection_data/Latex/pdf/1402.4029_3.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1402.4033_1.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.4033_5.pdf 8
../data/tablebank/Detection_data/Latex/pdf/1402.4033_6.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.4033_7.pdf 6
../data/tablebank/Detection_data/Latex/pdf/1402.4036_2.pdf 12
../data/tablebank/Detection_data/Latex/pdf/1402.4046_5.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1402.4046_6.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1402.4047_40.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1402.4047_41.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1402.4047_42.pdf 1
../data/tablebank/De

../data/tablebank/Detection_data/Latex/pdf/1402.4237_1.pdf 7
../data/tablebank/Detection_data/Latex/pdf/1402.4238_24.pdf 26
../data/tablebank/Detection_data/Latex/pdf/1402.4238_25.pdf 15
../data/tablebank/Detection_data/Latex/pdf/1402.4239_17.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.4239_18.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.4239_19.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.4239_35.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.4241_4.pdf 6
../data/tablebank/Detection_data/Latex/pdf/1402.4241_5.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1402.4243_4.pdf 5
../data/tablebank/Detection_data/Latex/pdf/1402.4243_5.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1402.4247_10.pdf 5
../data/tablebank/Detection_data/Latex/pdf/1402.4247_13.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1402.4247_14.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1402.4247_16.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1402.4247_5.pdf 1
../data/tabl

../data/tablebank/Detection_data/Latex/pdf/1402.4593_20.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1402.4593_24.pdf 2
../data/tablebank/Detection_data/Latex/pdf/1402.4593_26.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1402.4593_4.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1402.4593_8.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.4613_10.pdf 4
../data/tablebank/Detection_data/Latex/pdf/1402.4622_4.pdf 9
../data/tablebank/Detection_data/Latex/pdf/1402.4635_18.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1402.4640_6.pdf 1
../data/tablebank/Detection_data/Latex/pdf/1402.4649_3.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.4658_3.pdf 0
../data/tablebank/Detection_data/Latex/pdf/1402.4658_7.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1402.4666_6.pdf 3
../data/tablebank/Detection_data/Latex/pdf/1402.4675_13.pdf 7
../data/tablebank/Detection_data/Latex/pdf/1402.4687_5.pdf 5
../data/tablebank/Detection_data/Latex/pdf/1402.4692_11.pdf 4
../data/tablebank

In [24]:
# Filter
gt_latex = gt_subset[gt_subset["format"] == "Latex"]
print(len(gt_latex))
gt_latex.loc[:, "file"] = gt_latex[gt_latex["format"] == "Latex"]["file"].str.replace("images", "pdf")
gt_latex.loc[:, "file"] = gt_latex.loc[:, "file"].str.replace("jpg", "pdf")
gt_latex2 = gt_latex[~gt_latex["file"].isin(err_latex)]
print(len(gt_latex2))

5000
2991


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy [indexing.py:543]


### Word

In [7]:
# TODO uncomment
# word_links = pd.read_csv("../data/tablebank/Detection_data/Word/url.csv", sep=",")
# word_links = word_links[word_links["image_name"].isin(gt_subset["file"].apply(lambda x: get_file_name(x)))]
# word_links.head()

In [9]:
def get_files_word(url_row, err, s):
    # get
    filename = get_file_name(url_row["image_name"].replace("jpg", "pdf"))
    file1 = "../data/tablebank/Detection_data/Word/pdf/" + filename
    e = False
    if not os.path.exists(file1):
        try:
            data = urllib.request.urlretrieve(url_row["url"], "test.docx")
            docx2pdf.convert("test.docx", "test.pdf")
            inputpdf = PdfFileReader(open("test.pdf", "rb"))
            # store
            output = PdfFileWriter()
            output.addPage(inputpdf.getPage(url_row["page"]))
            with open(file1, "wb") as outputStream:
                output.write(outputStream)
        except Exception:
            e = True
    try:
        # compare
        file2 = "../data/tablebank/Detection_data/Word/images/" + url_row["image_name"]
        img2 = Image.open(file2)
        img1 = convert_from_path(file1, size=img2.size, fmt='jpg')[0]
        img1 = img1.resize(img2.size)
        sim = compare_img(img1, img2)
        print(file1 + " " + str(sim))
        if sim > s:
            e = True
    except Exception:
        e = True
    if e:
        err.append(file1)

In [10]:
def conv_url(link):
    p = link.split("/")
    return p[len(p)-1]

def replace_file_type(file_name, new_type):
    file_name_parts = file_name.split(".")
    return file_name.replace(file_name_parts[len(file_name_parts)-1], new_type)


def word2pdf(in_file, out_file):
    wdFormatPDF = 17
    word = comtypes.client.CreateObject('Word.Application')
    doc = word.Documents.Open(in_file)
    doc.SaveAs(out_file, FileFormat=wdFormatPDF)
    doc.Close()
    word.Quit()
    
def handler(signum, frame):
    raise Exception("failed")

t = conv_url("https://tps.org.uk/public/downloads/kZ6t_/%20Edward%20Dawes%20paper.docx")
print(t)
t = replace_file_type(t, "pdf")
print(t)

%20Edward%20Dawes%20paper.docx
%20Edward%20Dawes%20paper.pdf


In [13]:
def get_files_word2(url_row, err, s):
    signal.signal(signal.SIGALRM, handler)
    signal.alarm(15)
    # get
    filename = get_file_name(url_row["image_name"].replace("jpg", "pdf"))
    file1 = "../data/tablebank/Detection_data/Word/pdf/" + filename
    name_w = conv_url(url_row["url"])
    file_w = "../data/tablebank/Detection_data/Word/word/" + name_w
    file_p = "../data/tablebank/Detection_data/Word/pdf_complete/" + replace_file_type(name_w, "pdf")
    e = False
    try:
        # word
        if not os.path.exists(file_w):
            data = urllib.request.urlretrieve(url_row["url"], file_w)
            print(file_w)
        raise Exception("e")
        copyfile(file_w, "test.docx")
        # pdf
        if not os.path.exists(file_p): 
            docx2pdf.convert("test.docx", file_p)
        # page
        if not os.path.exists(file1):
            print(file1)
            inputpdf = PdfFileReader(open(file_p, "rb"))
            # store
            output = PdfFileWriter()
            output.addPage(inputpdf.getPage(url_row["page"]))
            with open(file1, "wb") as outputStream:
                output.write(outputStream)

            # compare
            file2 = "../data/tablebank/Detection_data/Word/images/" + url_row["image_name"]
            img2 = Image.open(file2)
            img1 = convert_from_path(file1, size=img2.size, fmt='jpg')[0]
            img1 = img1.resize(img2.size)
            sim = compare_img(img1, img2)
            print(file1 + " " + str(sim))
            if sim > s:
                e = True
    except Exception:
        e = True
    if e:
        err.append(file1)
        
def get_files_word3(url_row, err, s):
    signal.signal(signal.SIGALRM, handler)
    signal.alarm(15)
    name_w = conv_url(url_row["url"])
    file_w = "../data/tablebank/Detection_data/Word/word/" + name_w
    try:
    # word
        if not os.path.exists(file_w):
            data = urllib.request.urlretrieve(url_row["url"], file_w)
        print(file_w + " done")
    except Exception:
        print(file_w + " failed")

In [14]:
err_word = []
word_links.apply(lambda x: get_files_word3(x, err_word, HASH), axis=1)
print(len(err_word))

../data/tablebank/Detection_data/Word/word/%20Edward%20Dawes%20paper.docx done
../data/tablebank/Detection_data/Word/word/%20Edward%20Dawes%20paper.docx done
../data/tablebank/Detection_data/Word/word/%20Edward%20Dawes%20paper.docx done
../data/tablebank/Detection_data/Word/word/%20Edward%20Dawes%20paper.docx done
../data/tablebank/Detection_data/Word/word/%20Edward%20Dawes%20paper.docx done
../data/tablebank/Detection_data/Word/word/%20Edward%20Dawes%20paper.docx done
../data/tablebank/Detection_data/Word/word/%20Edward%20Dawes%20paper.docx done
../data/tablebank/Detection_data/Word/word/%20VNR%2014-I-9.docx done
../data/tablebank/Detection_data/Word/word/%28REVISED%29%20LEASE%2010%20PLUS%203%20AND%20PURCHASE%2044%20PLUS%203%20NEW%20VERSION.docx failed
../data/tablebank/Detection_data/Word/word/%5B%EA%B2%BD%ED%97%98%EB%B3%B4%EA%B3%A0%EC%84%9C%5D%EC%9D%B4%EB%8C%80%ED%91%9C_2010120172_Arrhus.docx failed
../data/tablebank/Detection_data/Word/word/%5BMS-DOM2E%5D-170314.docx done
../data/t

../data/tablebank/Detection_data/Word/word/%C3%85lesund+kompetanseregion+Kompetanseplan+2016+til+2018.docx failed
../data/tablebank/Detection_data/Word/word/%C3%85rsplan%208%20kl.%20tysk.docx failed
../data/tablebank/Detection_data/Word/word/%C3%89tkez%C3%A9si%20t%C3%A1j%C3%A9koztat%C3%B3%20a%20beiratkoz%C3%B3knak.docx done
../data/tablebank/Detection_data/Word/word/%C3%96nc%C3%BCp%C4%B1nar%20S%C4%B1nav%20kurulu%20komisyon%20karar%C4%B1.docx done
../data/tablebank/Detection_data/Word/word/%C3%9Cbungsbeispiel%20Bilanzierung%20Das%20Faltrad.docx failed
../data/tablebank/Detection_data/Word/word/%C3%9Cbungsbeispiel%20Bilanzierung%20Das%20Faltrad.docx failed
../data/tablebank/Detection_data/Word/word/%C3%9Cbungsbeispiel%20KORE%202015.docx failed
../data/tablebank/Detection_data/Word/word/%c4%87wiczenia-2015-2016-f.docx failed
../data/tablebank/Detection_data/Word/word/%c4%87wiczenia-2015-2016-f.docx failed
../data/tablebank/Detection_data/Word/word/%c4%87wiczenia-2015-2016-f.docx failed
..

../data/tablebank/Detection_data/Word/word/(20-7-2016)(11-37-45%20AM).docx failed
../data/tablebank/Detection_data/Word/word/(20-7-2016)(11-37-45%20AM).docx failed
../data/tablebank/Detection_data/Word/word/(20-7-2016)(11-37-45%20AM).docx failed
../data/tablebank/Detection_data/Word/word/(20-7-2016)(11-37-45%20AM).docx failed
../data/tablebank/Detection_data/Word/word/(20-7-2016)(11-37-45%20AM).docx failed
../data/tablebank/Detection_data/Word/word/(20-7-2016)(11-37-45%20AM).docx failed
../data/tablebank/Detection_data/Word/word/(20-7-2016)(11-37-45%20AM).docx failed
../data/tablebank/Detection_data/Word/word/(20131115.141308).wystapienie_pokontrolne_um_swidwin.docx failed
../data/tablebank/Detection_data/Word/word/(20131115.141308).wystapienie_pokontrolne_um_swidwin.docx failed
../data/tablebank/Detection_data/Word/word/(20131115.141308).wystapienie_pokontrolne_um_swidwin.docx failed
../data/tablebank/Detection_data/Word/word/(20131115.141308).wystapienie_pokontrolne_um_swidwin.docx f

../data/tablebank/Detection_data/Word/word/00-RFP-15-301_Application_Manual_2015-05-06.docx failed
../data/tablebank/Detection_data/Word/word/00-RFP-15-301_Application_Manual_2015-05-06.docx failed
../data/tablebank/Detection_data/Word/word/00-RFP-15-301_Application_Manual_2015-05-06.docx failed
../data/tablebank/Detection_data/Word/word/00-RFP-15-301_Application_Manual_2015-05-06.docx failed
../data/tablebank/Detection_data/Word/word/00-RFP-15-301_Application_Manual_2015-05-06.docx failed
../data/tablebank/Detection_data/Word/word/00-RFP-15-301_Application_Manual_2015-05-06.docx failed
../data/tablebank/Detection_data/Word/word/00-RFP-15-301_Application_Manual_2015-05-06.docx failed
../data/tablebank/Detection_data/Word/word/00-RFP-15-301_Application_Manual_2015-05-06.docx failed
../data/tablebank/Detection_data/Word/word/00-RFP-15-301_Application_Manual_2015-05-06.docx failed
../data/tablebank/Detection_data/Word/word/00-RFP-15-301_Application_Manual_2015-05-06.docx failed
../data/ta

../data/tablebank/Detection_data/Word/word/006-Regional-Dynamics-of-Southwest-Pacific.docx failed
../data/tablebank/Detection_data/Word/word/006-Regional-Dynamics-of-Southwest-Pacific.docx failed
../data/tablebank/Detection_data/Word/word/006-Regional-Dynamics-of-Southwest-Pacific.docx failed
../data/tablebank/Detection_data/Word/word/00604-06001-MP-Pengendalian-Dokumen-Rekaman-UJM.docx done
../data/tablebank/Detection_data/Word/word/00604-06001-MP-Pengendalian-Dokumen-Rekaman-UJM.docx done
../data/tablebank/Detection_data/Word/word/006s.docx done
../data/tablebank/Detection_data/Word/word/007%20-%20Seg.%20Plan%20Anual%20-%20Proyecto%20Cluster%20Hortifruticola%20Boyaca.docx failed
../data/tablebank/Detection_data/Word/word/007%20-%20Seg.%20Plan%20Anual%20-%20Proyecto%20Cluster%20Hortifruticola%20Boyaca.docx failed
../data/tablebank/Detection_data/Word/word/007%20-%20Seg.%20Plan%20Anual%20-%20Proyecto%20Cluster%20Hortifruticola%20Boyaca.docx failed
../data/tablebank/Detection_data/Word/

...

../data/tablebank/Detection_data/Word/word/1518_South_Johnstone_SS_sar2017.docx failed
../data/tablebank/Detection_data/Word/word/1519236_om-gns-mallen-v2.docx done
../data/tablebank/Detection_data/Word/word/152013.-II.-26.-EMMI-rendelet.docx done
../data/tablebank/Detection_data/Word/word/152013.-II.-26.-EMMI-rendelet.docx done
../data/tablebank/Detection_data/Word/word/152013.-II.-26.-EMMI-rendelet.docx done
../data/tablebank/Detection_data/Word/word/152013.-II.-26.-EMMI-rendelet.docx done
../data/tablebank/Detection_data/Word/word/152013.-II.-26.-EMMI-rendelet.docx done
../data/tablebank/Detection_data/Word/word/152013.-II.-26.-EMMI-rendelet.docx done
../data/tablebank/Detection_data/Word/word/152013.-II.-26.-EMMI-rendelet.docx done
../data/tablebank/Detection_data/Word/word/152013.-II.-26.-EMMI-rendelet.docx done
../data/tablebank/Detection_data/Word/word/152013.-II.-26.-EMMI-rendelet.docx done
../data/tablebank/Detection_data/Word/word/1520790926.docx done
../data/tablebank/Detect

../data/tablebank/Detection_data/Word/word/153371_Ase_eat_3_6_Tyohon_opastaminen_ja_ohjaaminen_asealalla_arviointilomake.docx failed
../data/tablebank/Detection_data/Word/word/153371_Ase_eat_3_6_Tyohon_opastaminen_ja_ohjaaminen_asealalla_arviointilomake.docx failed
../data/tablebank/Detection_data/Word/word/153371_Ase_eat_3_6_Tyohon_opastaminen_ja_ohjaaminen_asealalla_arviointilomake.docx failed
../data/tablebank/Detection_data/Word/word/153371_Ase_eat_3_6_Tyohon_opastaminen_ja_ohjaaminen_asealalla_arviointilomake.docx failed
../data/tablebank/Detection_data/Word/word/153371_Ase_eat_3_6_Tyohon_opastaminen_ja_ohjaaminen_asealalla_arviointilomake.docx failed
../data/tablebank/Detection_data/Word/word/1538841_about_the_gns-template.docx done
../data/tablebank/Detection_data/Word/word/1538_62b8f07b0b5b6c59a18745b3042caef9.docx failed
../data/tablebank/Detection_data/Word/word/1538_62b8f07b0b5b6c59a18745b3042caef9.docx failed
../data/tablebank/Detection_data/Word/word/1538_62b8f07b0b5b6c59a

../data/tablebank/Detection_data/Word/word/157704_100000Master_MUERGE_Horarios_semestre_primavera_curso_17_18_v01.docx failed
../data/tablebank/Detection_data/Word/word/158360Supplemental_Material.docx done
../data/tablebank/Detection_data/Word/word/158360Supplemental_Material.docx done
../data/tablebank/Detection_data/Word/word/1584_e6a11145a13fee22b98e64a86b60bba6.docx done
../data/tablebank/Detection_data/Word/word/1584_e6a11145a13fee22b98e64a86b60bba6.docx done
../data/tablebank/Detection_data/Word/word/1584_e6a11145a13fee22b98e64a86b60bba6.docx done
../data/tablebank/Detection_data/Word/word/159%20Test%20psicodiagnostico%20gestalt.docx failed
../data/tablebank/Detection_data/Word/word/159%20Test%20psicodiagnostico%20gestalt.docx failed
../data/tablebank/Detection_data/Word/word/1591_c024bda310695b12360a4a094518d528.docx done
../data/tablebank/Detection_data/Word/word/1593585281security_tender.docx done
../data/tablebank/Detection_data/Word/word/159783828.docx done
../data/tableban

../data/tablebank/Detection_data/Word/word/16-17-1st-Grade-Post-test-.docx failed
../data/tablebank/Detection_data/Word/word/16-17-1st-Grade-Post-test-.docx failed
../data/tablebank/Detection_data/Word/word/16-17-1st-Grade-Post-test-.docx failed
../data/tablebank/Detection_data/Word/word/16-17-1st-Grade-Post-test-.docx failed
../data/tablebank/Detection_data/Word/word/16-17_ap_studio_art_curriculum_guide.docx done
../data/tablebank/Detection_data/Word/word/16-17_ap_studio_art_curriculum_guide.docx done
../data/tablebank/Detection_data/Word/word/16-17_ap_studio_art_curriculum_guide.docx done
../data/tablebank/Detection_data/Word/word/16-17_middle_school_curriculum_guide.docx done
../data/tablebank/Detection_data/Word/word/16-17_middle_school_curriculum_guide.docx done
../data/tablebank/Detection_data/Word/word/16-17_middle_school_curriculum_guide.docx done
../data/tablebank/Detection_data/Word/word/16-17_middle_school_curriculum_guide.docx done
../data/tablebank/Detection_data/Word/word

../data/tablebank/Detection_data/Word/word/160629_Vpliv_posameznik_na_visino_obracunane_akontacije_dohodnine.docx failed
../data/tablebank/Detection_data/Word/word/160706%20Regulation%20of%20Teacher%20Accreditation%20Authorities.docx failed
../data/tablebank/Detection_data/Word/word/160706%20Regulation%20of%20Teacher%20Accreditation%20Authorities.docx failed
../data/tablebank/Detection_data/Word/word/160706%20Regulation%20of%20Teacher%20Accreditation%20Authorities.docx failed
../data/tablebank/Detection_data/Word/word/160706%20Regulation%20of%20Teacher%20Accreditation%20Authorities.docx failed
../data/tablebank/Detection_data/Word/word/160706%20Regulation%20of%20Teacher%20Accreditation%20Authorities.docx failed
../data/tablebank/Detection_data/Word/word/160706%20Regulation%20of%20Teacher%20Accreditation%20Authorities.docx failed
../data/tablebank/Detection_data/Word/word/160706%20Regulation%20of%20Teacher%20Accreditation%20Authorities.docx failed
../data/tablebank/Detection_data/Word/w

In [ ]:
err_word

In [ ]:
# Filter
gt_word = gt_subset[gt_subset["format"] == "Word"]
print(len(gt_word))
gt_word.loc[:, "file"] = gt_word["file"].str.replace("images", "pdf")
gt_word.loc[:, "file"] = gt_word.loc[:, "file"].str.replace("jpg", "pdf")
gt_word2 = gt_word[~gt_word["file"].isin(err_word)]
print(len(gt_word2))

In [ ]:
gt_word2.append(gt_latex2).to_csv("../data/tablebank/Detection_data/gt_subset_5000.csv", sep=";", index=0)